In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import os
path = '/content/gdrive/MyDrive/날씨빅콘'
os.chdir(path)

In [ ]:
import numpy as np
import pandas as pd
import logging
from sklearn.preprocessing import StandardScaler, PowerTransformer
from sklearn.metrics import mean_squared_error
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dropout, Dense, GRU, Bidirectional
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
train = pd.read_csv('./검증용/data/train_pre.csv')
test = pd.read_csv('./검증용/data/test_pre.csv')

In [ ]:
train_a = train[train['fc_year'] == 2021]
train_b = train[train['fc_year'] == 2022]
train_c = train[train['fc_year'] == 2023]

In [ ]:
class IPythonHandler(logging.Handler):
    def emit(self, record):
        log_entry = self.format(record)
        print(log_entry)

logger = logging.getLogger()
logger.setLevel(logging.INFO)
logger.handlers = []  # 기존 핸들러 제거
handler = IPythonHandler()
formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
handler.setFormatter(formatter)
logger.addHandler(handler)

# 가중치 계산 함수
def calculate_weights(dh, alpha=10, dh_max=240):
    weights = np.exp(-alpha * dh / dh_max)
    logging.info(f"가중치 계산: alpha={alpha}, weights={weights[:5]}")  # 첫 5개 가중치 출력
    return weights

# 데이터 준비 함수
def prepare_data(train, validation1, validation2, test, selected_columns, sequence_length, alpha=10):
    logging.info("데이터 준비 중...")
    train = train.sort_values(by=['basis_index','dh'], ascending=[True, False])
    validation1 = validation1.sort_values(by=['basis_index','dh'], ascending=[True, False])
    validation2 = validation2.sort_values(by=['basis_index','dh'], ascending=[True, False])
    test_sorted = test.sort_values(by=['basis_index','dh'], ascending=[True, False])

    train = train[selected_columns + ['dh', 'vv']]
    validation1 = validation1[selected_columns + ['dh', 'vv']]
    validation2 = validation2[selected_columns + ['dh', 'vv']]
    test_sorted = test_sorted[selected_columns + ['dh']]

    weights_train = calculate_weights(train['dh'], alpha).values  # weights를 배열로 변환

    X_train = train.drop(['vv', 'dh'], axis=1).values
    y_train = train['vv'].values

    X_val1 = validation1.drop(['vv', 'dh'], axis=1).values
    y_val1 = validation1['vv'].values

    X_val2 = validation2.drop(['vv', 'dh'], axis=1).values
    y_val2 = validation2['vv'].values

    X_test = test_sorted.drop(['dh'], axis=1).values

    X_train_seq, y_train_seq, weights_train_seq = create_sequences(X_train, y_train, weights_train, sequence_length)
    X_val1_seq, y_val1_seq, _ = create_sequences(X_val1, y_val1, np.zeros_like(y_val1), sequence_length)
    X_val2_seq, y_val2_seq, _ = create_sequences(X_val2, y_val2, np.zeros_like(y_val2), sequence_length)
    X_test_seq, _, _ = create_sequences(X_test, np.zeros(len(X_test)), np.zeros(len(X_test)), sequence_length, is_test=True)

    logging.info("데이터 준비 완료")
    return X_train_seq, y_train_seq, X_val1_seq, y_val1_seq, X_val2_seq, y_val2_seq, X_test_seq, weights_train_seq, test, sequence_length

def create_sequences(X, y, weights, sequence_length, is_test=False):
    X_seq, y_seq, weights_seq = [], [], []
    for i in range(len(X) - sequence_length + 1):
        X_seq.append(X[i:i+sequence_length])
        y_seq.append(y[i+sequence_length-1])
        weights_seq.append(weights[i+sequence_length-1])
    return np.array(X_seq), np.array(y_seq), np.array(weights_seq)

# 표준화 함수 (Yeo-Johnson 변환 후 표준화)
def standardize_data(X_train, X_val1, X_val2, X_test):
    logging.info("데이터 표준화 중...")
    scaler = StandardScaler()
    transformer = PowerTransformer(method='yeo-johnson')

    X_train_shape = X_train.shape
    X_val1_shape = X_val1.shape
    X_val2_shape = X_val2.shape
    X_test_shape = X_test.shape

    X_train = X_train.reshape(-1, X_train_shape[-1])
    X_val1 = X_val1.reshape(-1, X_val1_shape[-1])
    X_val2 = X_val2.reshape(-1, X_val2_shape[-1])
    X_test = X_test.reshape(-1, X_test_shape[-1])

    X_train = transformer.fit_transform(X_train)
    X_val1 = transformer.transform(X_val1)
    X_val2 = transformer.transform(X_val2)
    X_test = transformer.transform(X_test)

    X_train_scaled = scaler.fit_transform(X_train).reshape(X_train_shape)
    X_val1_scaled = scaler.transform(X_val1).reshape(X_val1_shape)
    X_val2_scaled = scaler.transform(X_val2).reshape(X_val2_shape)
    X_test_scaled = scaler.transform(X_test).reshape(X_test_shape)

    logging.info("데이터 표준화 완료")
    return X_train_scaled, X_val1_scaled, X_val2_scaled, X_test_scaled

# 계급 구간 설정 및 분류 함수
def classify_intervals(values):
    intervals = [
        (-np.inf, 0.1, 0),    # 0.1 미만 (무강수)
        (0.1, 0.2, 1),  # 0.1 이상 0.2 미만
        (0.2, 0.5, 2),  # 0.2 이상 0.5 미만
        (0.5, 1.0, 3),  # 0.5 이상 1.0 미만
        (1.0, 2.0, 4),  # 1.0 이상 2.0 미만
        (2.0, 5.0, 5),  # 2.0 이상 5.0 미만
        (5.0, 10.0, 6), # 5.0 이상 10.0 미만
        (10.0, 20.0, 7),# 10.0 이상 20.0 미만
        (20.0, 30.0, 8),# 20.0 이상 30.0 미만
        (30.0, np.inf, 9)# 30.0 이상
    ]

    classified = np.full_like(values, -999, dtype=np.int32)  # 초기값은 -999로 설정하고 int32로 명시적으로 지정
    for lower, upper, grade in intervals:
        mask = (values >= lower) & (values < upper)
        classified[mask] = grade

    return classified

# CSI 계산을 위한 텐서플로우 연산
def calculate_csi_tf(y_true, y_pred):
    H = tf.reduce_sum(tf.cast((y_true == y_pred) & (y_true != 0), tf.float32))
    F = tf.reduce_sum(tf.cast((y_true != y_pred) & (y_pred != 0), tf.float32))
    M = tf.reduce_sum(tf.cast((y_true != y_pred) & (y_pred == 0), tf.float32))
    CSI = H / (H + F + M + 1e-7)  # 0으로 나누는 것을 방지하기 위해 작은 값을 더함
    return CSI

# 배치 단위로 CSI와 RMSE를 결합한 손실함수 계산
@tf.function(reduce_retracing=True)
def custom_loss(y_true, y_pred, weights, beta):
    y_true = tf.cast(y_true, tf.float32)
    y_pred = tf.cast(y_pred, tf.float32)

    rmse = tf.sqrt(tf.reduce_mean(tf.square(y_true - y_pred)))

    y_true_class = tf.py_function(func=classify_intervals, inp=[y_true], Tout=tf.int32)
    y_pred_class = tf.py_function(func=classify_intervals, inp=[y_pred], Tout=tf.int32)
    csi = calculate_csi_tf(y_true_class, y_pred_class)

    combined_loss = rmse + beta * (1 - csi)
    return combined_loss

# 모델 학습 함수
def train_model(X_train, y_train, weights_train, X_val1, y_val1, X_val2, y_val2, sequence_length, batch_size, epochs=60, beta=1.0):
    logging.info("GRU 모델 학습 중...")

    model = Sequential([
        Bidirectional(LSTM(128, return_sequences=True, dropout=0.1, input_shape=(sequence_length, X_train.shape[2]))),
        Bidirectional(LSTM(64)),
        Dense(1)
    ])

    optimizer = tf.keras.optimizers.Adam(learning_rate=0.0007)

    # EarlyStopping 수동 구현을 위한 변수들
    best_val_loss = np.inf
    best_weights = None
    patience = 25
    wait = 0

    train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train, weights_train)).batch(batch_size)
    val1_dataset = tf.data.Dataset.from_tensor_slices((X_val1, y_val1)).batch(batch_size)
    val2_dataset = tf.data.Dataset.from_tensor_slices((X_val2, y_val2)).batch(batch_size)

    for epoch in range(epochs):
        logging.info(f'Epoch {epoch+1}/{epochs}')

        # Training loop
        for step, (x_batch_train, y_batch_train, w_batch_train) in enumerate(train_dataset):
            with tf.GradientTape() as tape:
                y_pred_train = model(x_batch_train, training=True)
                loss_value = custom_loss(y_batch_train, y_pred_train, w_batch_train, beta)

            grads = tape.gradient(loss_value, model.trainable_weights)
            optimizer.apply_gradients(zip(grads, model.trainable_weights))

        # Validation1 loop
        val1_pred = []
        val1_true = []
        for x_batch_val1, y_batch_val1 in val1_dataset:
            y_pred_val1 = model(x_batch_val1, training=False)
            val1_pred.append(y_pred_val1)
            val1_true.append(y_batch_val1)

        val1_pred = tf.concat(val1_pred, axis=0)
        val1_true = tf.concat(val1_true, axis=0)

        val1_rmse = mean_squared_error(val1_true.numpy(), val1_pred.numpy(), squared=False)
        val1_true_class = classify_intervals(val1_true.numpy().flatten())
        val1_pred_class = classify_intervals(val1_pred.numpy().flatten())
        val1_csi = calculate_csi_tf(val1_true_class, val1_pred_class).numpy()

        val1_loss = val1_rmse + beta * (1 - val1_csi)
        logging.info(f'Validation1 RMSE: {val1_rmse}, Validation1 CSI: {val1_csi}, Validation1 Loss: {val1_loss}')

        # Validation2 loop
        val2_pred = []
        val2_true = []
        for x_batch_val2, y_batch_val2 in val2_dataset:
            y_pred_val2 = model(x_batch_val2, training=False)
            val2_pred.append(y_pred_val2)
            val2_true.append(y_batch_val2)

        val2_pred = tf.concat(val2_pred, axis=0)
        val2_true = tf.concat(val2_true, axis=0)

        val2_rmse = mean_squared_error(val2_true.numpy(), val2_pred.numpy(), squared=False)
        val2_true_class = classify_intervals(val2_true.numpy().flatten())
        val2_pred_class = classify_intervals(val2_pred.numpy().flatten())
        val2_csi = calculate_csi_tf(val2_true_class, val2_pred_class).numpy()

        val2_loss = val2_rmse + beta * (1 - val2_csi)
        logging.info(f'Validation2 RMSE: {val2_rmse}, Validation2 CSI: {val2_csi}, Validation2 Loss: {val2_loss}')

        avg_val_rmse = (val1_rmse + val2_rmse) / 2
        avg_val_csi = (val1_csi + val2_csi) / 2
        avg_val_loss = (val1_loss + val2_loss) / 2

        logging.info(f'Average RMSE: {avg_val_rmse}, Average CSI: {avg_val_csi}, Average Loss: {avg_val_loss}')

        # Early stopping 체크
        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            best_weights = model.get_weights()
            wait = 0
        else:
            wait += 1
            if wait >= patience:
                logging.info(f'Early stopping at epoch {epoch+1}')
                model.set_weights(best_weights)
                break

    model.set_weights(best_weights)
    logging.info("GRU 모델 학습 완료")
    return model

# 성능 평가 함수
def evaluate_model(model, X_val1, y_val1, X_val2, y_val2, beta=1.0):
    logging.info("모델 성능 평가 중...")

    # Validation1
    y_val1_pred = model.predict(X_val1)
    val1_rmse = mean_squared_error(y_val1, y_val1_pred, squared=False)
    y_val1_class = classify_intervals(y_val1.flatten())
    y_val1_pred_class = classify_intervals(y_val1_pred.flatten())
    val1_csi = calculate_csi_tf(y_val1_class, y_val1_pred_class).numpy()
    val1_loss = val1_rmse + beta * (1 - val1_csi)

    # Validation2
    y_val2_pred = model.predict(X_val2)
    val2_rmse = mean_squared_error(y_val2, y_val2_pred, squared=False)
    y_val2_class = classify_intervals(y_val2.flatten())
    y_val2_pred_class = classify_intervals(y_val2_pred.flatten())
    val2_csi = calculate_csi_tf(y_val2_class, y_val2_pred_class).numpy()
    val2_loss = val2_rmse + beta * (1 - val2_csi)

    # 평균 성능 평가
    avg_rmse = (val1_rmse + val2_rmse) / 2
    avg_csi = (val1_csi + val2_csi) / 2
    avg_loss = (val1_loss + val2_loss) / 2

    results = {
        'model': model,
        'avg_rmse': avg_rmse,
        'avg_csi': avg_csi,
        'avg_loss': avg_loss
    }

    logging.info(f'Validation1 RMSE: {val1_rmse}, Validation1 CSI: {val1_csi}, Validation1 Loss: {val1_loss}')
    logging.info(f'Validation2 RMSE: {val2_rmse}, Validation2 CSI: {val2_csi}, Validation2 Loss: {val2_loss}')
    logging.info(f'Average RMSE: {avg_rmse}, Average CSI: {avg_csi}, Average Loss: {avg_loss}')
    logging.info("모델 성능 평가 완료")
    return results

# 테스트 셋 예측 함수
def predict_test_set(model, X_test, test, sequence_length):
    logging.info("테스트 셋 예측 중...")
    test_sorted = test.sort_values(by=['basis_index','dh_rank'], ascending=[True, False])
    test_sorted['BiLSTM_pred'] = 0  # 초기화

    predictions = model.predict(X_test)
    test_sorted.iloc[sequence_length-1:, test_sorted.columns.get_loc('BiLSTM_pred')] = predictions  # 시퀀스 길이만큼 예측 불가능한 부분을 0으로

    test_with_predictions = test_sorted.sort_index()  # 원래의 순서로 복원
    logging.info("테스트 셋 예측 완료")
    return test_with_predictions

# 전체 파이프라인 함수
def run_pipeline(train, validation1, validation2, test, selected_columns, sequence_length=50, batch_size=64, alpha=4, beta=20):
    # 데이터 준비
    X_train_seq, y_train_seq, X_val1_seq, y_val1_seq, X_val2_seq, y_val2_seq, X_test_seq, weights_train_seq, test_original, sequence_length = prepare_data(train, validation1, validation2, test, selected_columns, sequence_length, alpha)

    # 데이터 표준화
    X_train_scaled, X_val1_scaled, X_val2_scaled, X_test_scaled = standardize_data(X_train_seq, X_val1_seq, X_val2_seq, X_test_seq)

    # 모델 학습
    model = train_model(X_train_scaled, y_train_seq, weights_train_seq, X_val1_scaled, y_val1_seq, X_val2_scaled, y_val2_seq, sequence_length, batch_size, beta=beta)

    # 성능 평가
    result = evaluate_model(model, X_val1_scaled, y_val1_seq, X_val2_scaled, y_val2_seq, beta)

    # 테스트 셋 예측
    test_with_predictions = predict_test_set(model, X_test_scaled, test_original, sequence_length)

    return result, test_with_predictions

In [ ]:
selected_col = [
    'v01', 'v02', 'v03', 'v04', 'v05', 'v06', 'v07', 'v08', 'v09',
    'v00_ind', 'v01_ind', 'v02_ind', 'v03_ind', 'v04_ind', 'v05_ind', 'v06_ind', 'v07_ind', 'v08_ind', 'v09_ind',
    'v_expect'
]

# 파이프라인 실행
results, test_pred_num = run_pipeline(train_c, train_a, train_b, test, selected_col, alpha=4, beta=40, sequence_length=50, batch_size=64)

results

2024-06-27 10:53:54,811 - INFO - 데이터 준비 중...
2024-06-27 10:53:55,707 - INFO - 가중치 계산: alpha=4, weights=964109     0.951229
988534     0.951229
1012916    0.951229
1037254    0.951229
1061447    0.951229
Name: dh, dtype: float64
2024-06-27 10:54:01,631 - INFO - 데이터 준비 완료
2024-06-27 10:54:01,634 - INFO - 데이터 표준화 중...
2024-06-27 11:00:46,004 - INFO - 데이터 표준화 완료
2024-06-27 11:00:46,030 - INFO - GRU 모델 학습 중...
2024-06-27 11:01:03,910 - INFO - Epoch 1/60
2024-06-27 11:01:10,284 - WARNING - 5 out of the last 5 calls to <function _BaseOptimizer._update_step_xla at 0x7e25d4a3f640> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings could be due to (1) creating @tf.function repeatedly in a loop, (2) passing tensors with different shapes, (3) passing Python objects instead of tensors. For (1), please define your @tf.function outside of the loop. For (2), @tf.function has reduce_retracing=True option that can avoid unnecessary retracing. For (3), please refer

{'model': <keras.src.engine.sequential.Sequential at 0x7e25d4d53c10>,
 'avg_rmse': 4.1271367923796545,
 'avg_csi': 0.08495622873306274,
 'avg_loss': 40.72888749404554}

In [ ]:
test_pred_num['class_interval'] = classify_intervals(test_pred_num['BiLSTM_pred'].round(2))
test_pred_num['class_interval'].value_counts()

class_interval
0    99424
5    10450
4     4000
3     2943
2     2505
6     1571
1     1069
7       38
Name: count, dtype: int64

In [ ]:
sub =  pd.read_csv('./검증용/rainfall_test/rainfall_test.csv')

In [ ]:
sub['rainfall_test.class_interval'] = sub['rainfall_test.class_interval'].combine_first(
    test_pred_num['class_interval'].where(sub['rainfall_test.class_interval'].isna())
)

In [ ]:
sub['rainfall_test.class_interval'] = sub['rainfall_test.class_interval'].astype(int)

In [ ]:
sub['rainfall_test.class_interval'].value_counts()

rainfall_test.class_interval
 0      99294
 5      10450
 4       4000
 3       2943
 2       2505
 6       1571
 1       1069
-999      130
 7         38
Name: count, dtype: int64

In [ ]:
sub['rainfall_test.class_interval'].value_counts()/len(sub)

rainfall_test.class_interval
 0      0.813885
 5      0.085656
 4      0.032787
 3      0.024123
 2      0.020533
 6      0.012877
 1      0.008762
-999    0.001066
 7      0.000311
Name: count, dtype: float64

In [ ]:
sub.to_csv('240210_BiLSTM.csv', index=False)